In [1]:
import os

In [2]:
os.getcwd()

'/home/tripti/FYP_2024'

In [3]:
%pip install scikit-learn

/bin/bash: /home/tripti/miniconda3/envs/tensorflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle
import numpy as np

In [4]:
%pip install tensorflow

/bin/bash: /home/tripti/miniconda3/envs/tensorflow/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers, Model
import numpy as np

2024-11-27 16:49:08.601198: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 16:49:08.620428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732706348.642737   15769 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732706348.649158   15769 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 16:49:08.672080: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
pickle_MCI_Features_file_path = r'./feature_extraction/MCI_final_features_all_1.pkl'
with open(pickle_MCI_Features_file_path, 'rb') as file:
    MCI_funct_features = pickle.load(file)
MCI_funct_features.shape
MCI_funct_features = np.array(MCI_funct_features)

In [5]:
MCI_funct_features.shape

(53, 13456)

In [6]:
pickle_MCI_Features_file_path = r'feature_extraction/cat12_structural_features.pkl'
with open(pickle_MCI_Features_file_path, 'rb') as file:
    MCI_Struct_features = pickle.load(file)
MCI_Struct_features.shape
MCI_Struct_features = np.array(MCI_Struct_features)

In [7]:
MCI_Struct_features.shape

(53, 169, 205, 169)

In [8]:
# Define the same file path
file_path = r'feature_extraction/Structal_cat_fpickle.pkl'

# Load the NumPy array from the file
with open(file_path, "rb") as file:  # 'rb' means read binary
    loaded_structural_features = pickle.load(file)

print("structural_features loaded successfully!")
print("Shape of loaded array:", loaded_structural_features.shape)


structural_features loaded successfully!
Shape of loaded array: (40, 169, 205, 169)


In [9]:
pickle_AD_Features_file_path = r"feature_extraction/func_features_AD.pkl"
with open(pickle_AD_Features_file_path, 'rb') as file:
    AD_funct_features = pickle.load(file)
AD_funct_features = np.array(AD_funct_features)
AD_funct_features.shape

(40, 13456)

In [10]:
MCI_funct_features.shape

(53, 13456)

In [11]:
MCI_Struct_features.shape


(53, 169, 205, 169)

In [12]:
AD_funct_features.shape


(40, 13456)

In [13]:
loaded_structural_features.shape

(40, 169, 205, 169)

In [14]:

# Data Shapes
AD_Struct_features = loaded_structural_features
# Labels
MCI_labels = np.zeros(53)  # Label 0 for MCI
AD_labels = np.ones(40)    # Label 1 for AD

# Combine functional and structural features
funct_features = np.vstack((MCI_funct_features, AD_funct_features))
struct_features = np.vstack((MCI_Struct_features, AD_Struct_features))
labels = np.hstack((MCI_labels, AD_labels))

In [15]:
import numpy as np

# Assuming you have loaded your SMRI data into variables
# MCI_Struct_features and AD_Struct_features

# Combine MCI and AD structural features
SMRI_data = np.concatenate([MCI_Struct_features, AD_Struct_features], axis=0)
SMRI_labels = np.concatenate([
    np.zeros(len(MCI_Struct_features)),  # Label 0 for MCI
    np.ones(len(AD_Struct_features))     # Label 1 for AD
], axis=0)

# Add a channel dimension
SMRI_data = np.expand_dims(SMRI_data, axis=-1)  # Shape becomes (num_samples, 169, 205, 169, 1)

In [16]:
# Assuming you have loaded your fMRI data into variables
# MCI_funct_features and AD_funct_features

# Combine MCI and AD functional features
fMRI_data = np.concatenate([MCI_funct_features, AD_funct_features], axis=0)

# Reshape and add channel dimension
fMRI_data = fMRI_data.reshape(-1, 116, 116)
fMRI_data = np.expand_dims(fMRI_data, axis=-1)  # Shape becomes (num_samples, 116, 116, 1)

In [17]:
labels = np.concatenate([
    np.zeros(len(MCI_funct_features)),  # Label 0 for MCI
    np.ones(len(AD_funct_features))     # Label 1 for AD
], axis=0)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
# Split data into training and testing sets
X_train_3d, X_test_3d, X_train_2d, X_test_2d, y_train, y_test = train_test_split(
    SMRI_data,
    fMRI_data,
    labels,
    test_size=0.2,
    random_state=42
)

In [20]:
# Discriminator labels (real images are labeled as 1)
real_labels_train = np.ones((X_train_3d.shape[0], 1))
real_labels_test = np.ones((X_test_3d.shape[0], 1))

# For simplicity, use real labels for reconstructed images
disc_targets_3d_train = real_labels_train
disc_targets_2d_train = real_labels_train

disc_targets_3d_test = real_labels_test
disc_targets_2d_test = real_labels_test

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

def build_encoder_3d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Conv3D(64, (3, 3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Conv3D(128, (3, 3, 3), activation='relu', padding='same')(x)
    x = layers.Flatten()(x)
    latent = layers.Dense(256, activation='relu')(x)
    encoder = Model(inputs, latent, name='encoder_3d')
    return encoder

In [22]:
def build_encoder_2d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.Flatten()(x)
    latent = layers.Dense(256, activation='relu')(x)
    encoder = Model(inputs, latent, name='encoder_2d')
    return encoder

In [23]:
def build_decoder_3d(latent_dim, output_shape):
    latent_inputs = Input(shape=(latent_dim,))
    x = layers.Dense(np.prod(output_shape)//8, activation='relu')(latent_inputs)
    x = layers.Reshape((output_shape[0]//8, output_shape[1]//8, output_shape[2]//8, 8))(x)
    x = layers.Conv3DTranspose(128, (3, 3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv3DTranspose(64, (3, 3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv3DTranspose(32, (3, 3, 3), strides=2, padding='same', activation='relu')(x)
    outputs = layers.Conv3DTranspose(1, (3, 3, 3), padding='same', activation='sigmoid')(x)
    decoder = Model(latent_inputs, outputs, name='decoder_3d')
    return decoder

In [24]:
def build_decoder_2d(latent_dim, output_shape):
    latent_inputs = Input(shape=(latent_dim,))
    x = layers.Dense((output_shape[0]//4)*(output_shape[1]//4)*64, activation='relu')(latent_inputs)
    x = layers.Reshape((output_shape[0]//4, output_shape[1]//4, 64))(x)
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='relu')(x)
    outputs = layers.Conv2DTranspose(1, (3, 3), padding='same', activation='sigmoid')(x)
    decoder = Model(latent_inputs, outputs, name='decoder_2d')
    return decoder

In [25]:
def build_discriminator_3d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv3D(32, (3, 3, 3), activation='relu')(inputs)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Conv3D(64, (3, 3, 3), activation='relu')(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    discriminator = Model(inputs, outputs, name='discriminator_3d')
    return discriminator

In [26]:
def build_discriminator_2d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    discriminator = Model(inputs, outputs, name='discriminator_2d')
    return discriminator

In [27]:
def build_classifier(input_dim, num_classes):
    inputs = Input(shape=(input_dim,))
    x = layers.Dense(128, activation='relu')(inputs)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    classifier = Model(inputs, outputs, name='classifier')
    return classifier

In [28]:
# Input shapes
input_shape_3d = (169, 205, 169, 1)
input_shape_2d = (116, 116, 1)
latent_dim = 256
num_classes = 2

# Build models
encoder_3d = build_encoder_3d(input_shape_3d)
encoder_2d = build_encoder_2d(input_shape_2d)
decoder_3d = build_decoder_3d(latent_dim, input_shape_3d[:-1])
decoder_2d = build_decoder_2d(latent_dim, input_shape_2d[:-1])
discriminator_3d = build_discriminator_3d(input_shape_3d)
discriminator_2d = build_discriminator_2d(input_shape_2d)
# Build classifier that accepts concatenated latent vectors
classifier = build_classifier(latent_dim * 2, num_classes)

# Inputs
input_3d = Input(shape=input_shape_3d)
input_2d = Input(shape=input_shape_2d)

# Encoding
latent_3d = encoder_3d(input_3d)
latent_2d = encoder_2d(input_2d)

# Concatenate the two latent representations
combined_latent = layers.Concatenate()([latent_3d, latent_2d])

# Decoding
reconstructed_3d = decoder_3d(latent_3d)
reconstructed_2d = decoder_2d(latent_2d)

# Discriminator outputs
disc_output_3d = discriminator_3d(reconstructed_3d)
disc_output_2d = discriminator_2d(reconstructed_2d)

# Get classification output
classification_output = classifier(combined_latent)

# Define the combined model
model = Model(
    inputs=[input_3d, input_2d],
    outputs=[
        reconstructed_3d,
        reconstructed_2d,
        disc_output_3d,
        disc_output_2d,
        classification_output
    ]
)

I0000 00:00:1732706409.872513   15769 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 45811 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:4b:00.0, compute capability: 8.6


ValueError: The total size of the tensor must be unchanged. Received: input_shape=(np.int64(731875),), target_shape=(21, 25, 21, 8)

In [ ]:
# Loss weights
lambda_reconstruction = 1.0
lambda_adversarial = 0.1
lambda_classification = 1.0

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'decoder_3d': 'mse',
        'decoder_2d': 'mse',
        'discriminator_3d': 'binary_crossentropy',
        'discriminator_2d': 'binary_crossentropy',
        'classifier': 'sparse_categorical_crossentropy'
    },
    loss_weights={
        'decoder_3d': lambda_reconstruction,
        'decoder_2d': lambda_reconstruction,
        'discriminator_3d': lambda_adversarial,
        'discriminator_2d': lambda_adversarial,
        'classifier': lambda_classification
    },
    metrics={
        'classifier': 'accuracy'
    }
)

In [ ]:
# Discriminator labels (real images are labeled as 1)
real_labels = np.ones((SMRI_data.shape[0], 1))
fake_labels = np.zeros((SMRI_data.shape[0], 1))

# For simplicity, use real labels for reconstructed images (you can adjust as needed)
disc_targets_3d = real_labels
disc_targets_2d = real_labels

In [ ]:
# Train the model
history = model.fit(
    [X_train_3d, X_train_2d],
    {
        'decoder_3d': X_train_3d,
        'decoder_2d': X_train_2d,
        'discriminator_3d': disc_targets_3d_train,
        'discriminator_2d': disc_targets_2d_train,
        'classifier': y_train
    },
    epochs=20,
    batch_size=8,
    validation_data=(
        [X_test_3d, X_test_2d],
        {
            'decoder_3d': X_test_3d,
            'decoder_2d': X_test_2d,
            'discriminator_3d': disc_targets_3d_test,
            'discriminator_2d': disc_targets_2d_test,
            'classifier': y_test
        }
    )
)

In [ ]:
# Evaluate the classifier accuracy on the test set
evaluation = model.evaluate(
    [X_test_3d, X_test_2d],
    {
        'decoder_3d': X_test_3d,
        'decoder_2d': X_test_2d,
        'discriminator_3d': disc_targets_3d_test,
        'discriminator_2d': disc_targets_2d_test,
        'classifier': y_test
    },
    verbose=0
)

# Get the index of the classification loss and accuracy (depends on the order of outputs)
classification_accuracy = evaluation[model.metrics_names.index('classifier_accuracy')]

print(f'Classifier Test Accuracy: {classification_accuracy * 100:.2f}%')